# EDSA Movie Recommendation Challenge

# Introduction

## Context

In today’s technology driven world, recommender systems are socially and economically critical for ensuring that individuals can make appropriate choices surrounding the content they engage with on a daily basis. One application where this is especially true surrounds movie content recommendations; where intelligent algorithms can help viewers find great titles from tens of thousands of options.

## Problem Statement
With this context, EDSA is challenging you to construct a recommendation algorithm based on content or collaborative filtering, capable of accurately predicting how a user will rate a movie they have not yet viewed based on their historical preferences.

Providing an accurate and robust solution to this challenge has immense economic potential, with users of the system being exposed to content they would like to view or purchase - generating revenue and platform affinity.

## Evaluation
The evaluation metric for this competition is Root Mean Square Error. Root Mean Square Error (RMSE) is commonly used in regression analysis and forecasting, and measures the standard deviation of the residuals arising between predicted and actual observed values for a modelling process.

# Data Exploration

## Installing and Importing packages
An importang module we'll need to install and import here is turicreate. The following explanation is taken directly from the module's documentation page.

Turi Create simplifies the development of custom machine learning models. You don’t have to be a machine learning expert to add recommendations, object detection, image classification, image similarity or activity classification to your app.

1. Easy-to-use: Focus on tasks instead of algorithms
2. Visual: Built-in, streaming visualizations to explore your data
3. Flexible: Supports text, images, audio, video and sensor data
4. Fast and Scalable: Work with large datasets on a single machine
5. Ready To Deploy: Export models to Core ML for use in iOS, macOS, watchOS, and tvOS apps

In [3]:
!pip install -U turicreate

  Using cached turicreate-6.0.tar.gz (1.9 kB)
  Running setup.py clean for turicreate
Failed to build turicreate
    Running setup.py install for turicreate: started
    Running setup.py install for turicreate: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\MZEE\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\MZEE\\AppData\\Local\\Temp\\pip-install-yr6c_c4v\\turicreate\\setup.py'"'"'; __file__='"'"'C:\\Users\\MZEE\\AppData\\Local\\Temp\\pip-install-yr6c_c4v\\turicreate\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\MZEE\AppData\Local\Temp\pip-wheel-zi4pqi0d'
       cwd: C:\Users\MZEE\AppData\Local\Temp\pip-install-yr6c_c4v\turicreate\
  Complete output (31 lines):
  running bdist_wheel
  running build
  installing to build\bdist.win-amd64\wheel
  running install
  
  
  
          TURICREATE ERROR
  
          If you see this message, pip install did not find an available binary package
          for your system.
  
          Supported Platforms:
              * macOS 10.12

In [4]:
import pandas as pd
import numpy as np
import turicreate

ModuleNotFoundError: No module named 'turicreate'

## Loading data files

In [5]:
path = '/content/drive/My Drive/Projects/recommendation_engine/data/train.csv'
ratings_train = pd.read_csv(path, verbose = False)

In [6]:
ratings_train.head()

,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837


In [7]:
ratings_train.pop('timestamp')

0           1518349992
1           1206238739
2           1076215539
3           1423042565
4            833375837
               ...    
10000033    1521235092
10000034    1002580977
10000035    1227674807
10000036    1479921530
10000037     858984862
Name: timestamp, Length: 10000038, dtype: int64

In [8]:
n_users = ratings_train['userId'].nunique()
n_items = ratings_train['movieId'].nunique()

print('Number of Users: '+ str(n_users))
print('Number of Movies: '+str(n_items))

Number of Users: 162541
Number of Movies: 48213


# Data preprocessing

In [9]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(ratings_train, test_size = 0.25)

In [10]:
train_data = turicreate.load_sframe(train_data)
test_data = turicreate.load_sframe(test_data)

In [11]:
train_data

userId,movieId,rating
83779,7143,2.0
161339,18,4.0
88430,3615,3.0
127493,1252,5.0
104451,2110,2.0
68899,21,4.0
40265,46653,3.0
74683,539,4.0
115053,593,5.0
22022,480,4.0


# Base Model (Popularity Recommendation Engine)
Here we implement a simple model that makes recommendations using item popularity. When a target is provided, popularity is computed using the item's mean target value. When the target column contains ratings, as in our case, the model computes the mean rating for each item and uses this to rank items for recommendations.

## Implementation

In [12]:
popularity_model = turicreate.popularity_recommender.create(train_data, user_id='userId', item_id='movieId', target='rating')

Preparing data set.

Data has 7500028 observations with 162532 users and 44596 items.

Data prepared in: 10.8782s

7500028 observations to process; with 44596 unique items.

In [ ]:
filename = 'popular_model.sav'
pickle.dump(popularity_model, open(filename, 'wb'))

## Model Evaluation

In [13]:
popularity_model.evaluate(test_data, metric ='rmse', target = 'rating')


Overall RMSE: 0.9617601791978634

Per User RMSE (best)
+--------+------+-------+
| userId | rmse | count |
+--------+------+-------+
| 26012  | 0.0  |   1   |
+--------+------+-------+
[1 rows x 3 columns]


Per User RMSE (worst)
+--------+--------------------+-------+
| userId |        rmse        | count |
+--------+--------------------+-------+
| 136007 | 3.7786820767275806 |   1   |
+--------+--------------------+-------+
[1 rows x 3 columns]


Per Item RMSE (best)
+---------+------+-------+
| movieId | rmse | count |
+---------+------+-------+
|  182305 | 0.0  |   1   |
+---------+------+-------+
[1 rows x 3 columns]


Per Item RMSE (worst)
+---------+------+-------+
| movieId | rmse | count |
+---------+------+-------+
|  187921 | 4.5  |   1   |
+---------+------+-------+
[1 rows x 3 columns]



{'rmse_by_item': Columns:
 	movieId	int
 	rmse	float
 	count	int
 
 Rows: 31747
 
 Data:
 +---------+---------------------+-------+
 | movieId |         rmse        | count |
 +---------+---------------------+-------+
 |   4441  |  0.7392438579261119 |   36  |
 |  159407 |         0.5         |   1   |
 |  128606 |  0.853365466012948  |   19  |
 |  85892  | 0.33333333333333326 |   1   |
 |   7899  |  0.7619875327064086 |   16  |
 |  113767 |  1.4142135623730951 |   2   |
 |  182611 |  3.0336071412000063 |   1   |
 |  133925 |         0.0         |   1   |
 |  74426  |  1.3974182544169722 |   10  |
 |  144280 |  0.2999999999999998 |   1   |
 +---------+---------------------+-------+
 [31747 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'rmse_by_user': Columns:
 	userId	int
 	rmse	float
 	count	int
 
 Rows: 159317
 
 Data:
 +--------+--------------------+-------+
 | userId |        rms

The overall RMSE from this simple model is 0.96. Pretty good for a simple model.

## Making Movie Recommendations

In [19]:
# for two users recommend 5 movies
popularity_model.recommend(users = [133, 3567], k = 5)

userId,movieId,score,rank
133,127345,5.0,1
133,146810,5.0,2
133,105144,5.0,3
133,178251,5.0,4
133,160339,5.0,5
3567,127345,5.0,1
3567,146810,5.0,2
3567,105144,5.0,3
3567,178251,5.0,4
3567,160339,5.0,5


For example for user with userId 133, the 5 best recommendations would be the movies shown in the table above. Note that these are the same for the user with userId 3567. This is because the popularity recommendation model does not discriminate users based on preference, all users are simply recommended the same movies.